# Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по правилам:

- если источник traffic_source равен Yandex или Google, то в source_type ставится organic;
- для источников paid и email из России ставим ad;
- для источников paid и email не из России ставим other;
- все остальные варианты берём из traffic_source без изменений.

In [3]:
import pandas as pd

# Загрузка данных из visit_logs.csv
visit_logs_df = pd.read_csv('visit_log.csv', sep=';')

# Функция для определения типа источника
def determine_source_type(row):
    traffic_source = row['traffic_source'].lower()
    region = row['region']

    if traffic_source in ['yandex', 'google']:
        return 'organic'
    elif traffic_source in ['paid', 'email']:
        if region == 'Russia':
            return 'ad'
        else:
            return 'other'
    else:
        return traffic_source

# Применение функции к DataFrame
visit_logs_df['source_type'] = visit_logs_df.apply(determine_source_type, axis=1)

# Вывод результата
print(visit_logs_df)

        timestamp    visit_id                                 url   region  \
0      1549980692  e3b0c44298  https://host.ru/3c19b4ef7371864fa3   Russia   
1      1549980704  6e340b9cff  https://host.ru/c8d9213a31839f9a3a   Russia   
2      1549980715  96a296d224  https://host.ru/b8b58337d272ee7b15   Russia   
3      1549980725  709e80c884  https://host.ru/b8b58337d272ee7b15   Russia   
4      1549980736  df3f619804  https://host.ru/b8b58337d272ee7b15   Russia   
...           ...         ...                                 ...      ...   
18933  1550094288  57e5ba8560  https://host.ru/c2382eb3d6afc8d0f3  Belarus   
18934  1550094296  6f9389ec1b  https://host.ru/f1eb4601740d627ab0   Russia   
18935  1550094308  e8cf2eb8e6  https://host.ru/a5dda93e70318570c0  Belarus   
18936  1550094314  79530b9a67  https://host.ru/6fda01ec57f23abc9e   Russia   
18937  1550094323  b3b634f824  https://host.ru/39fa884393666d45fc   Russia   

          user_id traffic_source source_type  
0      b1613cc09

# Задание 2

В файле URLs.txt содержатся URL страниц новостного сайта. Вам нужно отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри URL конструкцию: /, затем 8 цифр, затем дефис. Выполните действия:

Прочитайте содержимое файла с датафрейм.
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствие с заданным шаблоном.

In [4]:
import pandas as pd

# Чтение содержимого файла urls.txt как .csv, т.к. там по факту есть заголовок и одна колонка
urls_df = pd.read_csv('URLs.txt')

# Фильтрация страниц с текстами новостей
# Регулярное выражение: /[0-9]{8}- для поиска 8 цифр после /
news_urls_df = urls_df[urls_df['url'].str.contains(r'/[0-9]{8}-', regex=True)]

# Вывод результата
print(news_urls_df)

                                                  url
3   /politics/36188461-s-marta-zhizn-rossiyan-susc...
4   /world/36007585-tramp-pridumal-kak-reshit-ukra...
5   /science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6   /video/36001498-poyavilis-pervye-podrobnosti-g...
7   /world/36007585-tramp-pridumal-kak-reshit-ukra...
..                                                ...
88  /cis/35984145-kreml-prokommentiroval-soobschen...
89  /video/36071019-olimpiyskie-obekty-rio-prevrat...
90  /science/36151301-nazvano-posledstvie-zloupotr...
91  /incidents/36027330-vospitatelnitsu-zatravili-...
92  /world/36103095-dominikanskih-zhurnalistov-ubi...

[84 rows x 1 columns]


# Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значениями столбца timestamp для данного значения userId.

In [ ]:
import pandas as pd

# Загрузка данных из ratings.csv
ratings_df = pd.read_csv('ratings.csv')

# Группировка по userId и подсчет количества оценок
user_rating_counts = ratings_df.groupby('userId').size()

# Фильтрация пользователей с более чем 100 оценками
active_users = user_rating_counts[user_rating_counts > 100].index

# Получение данных только для активных пользователей
active_users_df = ratings_df[ratings_df['userId'].isin(active_users)]

# Подсчет минимального и максимального timestamp для каждого пользователя
lifetime_df = active_users_df.groupby('userId')['timestamp'].agg(['min', 'max'])

# Вычисление времени жизни для каждого пользователя
lifetime_df['lifetime'] = lifetime_df['max'] - lifetime_df['min']

# Вычисление среднего времени жизни
average_lifetime = lifetime_df['lifetime'].mean()

# Вывод результата; предполагаю, что у меня миллисекунды получились в average_lifetime
print(f'Среднее время жизни пользователей с более чем 100 оценками: {average_lifetime/1000/60/60} часов')

Среднее время жизни пользователей с более чем 100 оценками: 11.133474291559 часов


# Задание 4

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

Нужно сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;
- аналогичную таблицу по типам выручки с указанием адреса клиента.


In [ ]:
import pandas as pd

# Данные по типам перевозок
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1',
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

# Объединение таблиц по client_id без потери данных
merged_revenue = rzd.merge(auto, on='client_id', how='outer').merge(air, on='client_id', how='outer')

# Таблица с выручкой без адреса
print("Таблица с выручкой без адреса:")
print(merged_revenue)

# Объединение с адресами клиентов
merged_with_addresses = merged_revenue.merge(client_base, on='client_id', how='outer')

# Таблица с выручкой с адресом
print("\nТаблица с выручкой с адресом:")
print(merged_with_addresses)


Таблица с выручкой без адреса:
   client_id  rzd_revenue  auto_revenue  air_revenue
0        111       1093.0           NaN          NaN
1        112       2810.0           NaN          NaN
2        113      10283.0       57483.0          NaN
3        114       5774.0          83.0          NaN
4        115        981.0         912.0         81.0
5        116          NaN        4834.0          4.0
6        117          NaN          98.0         13.0
7        118          NaN           NaN        173.0

Таблица с выручкой с адресом:
   client_id  rzd_revenue  auto_revenue  air_revenue          address
0        111       1093.0           NaN          NaN  Комсомольская 4
1        112       2810.0           NaN          NaN   Энтузиастов 8а
2        113      10283.0       57483.0          NaN  Левобережная 1а
3        114       5774.0          83.0          NaN          Мира 14
4        115        981.0         912.0         81.0        ЗЖБИиДК 1
5        116          NaN        4834.0  